# Lab 1 Part II: UCI-HAR 1D CNN

## Imports

In [ ]:
from pathlib import Path
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Input, Conv1D, MaxPool1D, Flatten, Dense, Activation
from keras.utils import get_file
from keras.utils import to_categorical

## Load and format UCI-HAR dataset (raw data)

In [ ]:
dataset_path = get_file(None, "https://archive.ics.uci.edu/ml/machine-learning-databases/00240/UCI%20HAR%20Dataset.zip", extract=True, file_hash="53e099237392e0b9602f8c38f578bd8f") # Download, cache and extract UCI-HAR
dataset_dir = Path(dataset_path).parent

def load_set(dataset_dir, part: str): # Load separate sensor signals and combine them into a single array, load labels separately
    data = np.hstack([np.loadtxt(dataset_dir/'UCI HAR Dataset'/part/'Inertial Signals'/f'{sensor}_{axis}_{part}.txt')
                for sensor in ('body_acc', 'body_gyro', 'total_acc')
                    for axis in ('x', 'y', 'z')]).reshape((-1, 128, 9))
    labels = to_categorical(np.loadtxt(dataset_dir/'UCI HAR Dataset'/part/f'y_{part}.txt') - 1)
    return data, labels

x_train, y_train = load_set(dataset_dir, 'train')
x_test, y_test = load_set(dataset_dir, 'test')

## Export complete test dataset (2947 vectors)

In [ ]:
np.savetxt('x_test_uci-har.csv', x_test.reshape((x_test.shape[0], -1)), delimiter=',', fmt='%s')
np.savetxt('y_test_uci-har.csv', y_test, delimiter=',', fmt='%s')

## Export small test dataset (250 vectors)

In [ ]:
x_test_250 = x_test[0:250]
y_test_250 = y_test[0:250]
np.savetxt('x_test_uci-har_250.csv', x_test_250.reshape((x_test_250.shape[0], -1)), delimiter=',', fmt='%s')
np.savetxt('y_test_uci-har_250.csv', y_test_250, delimiter=',', fmt='%s')

## Build model

In [ ]:
model = Sequential()
model.add(Input(shape=(128, 9)))
model.add(Conv1D(filters=2, kernel_size=3, activation='relu'))
model.add(Flatten())
model.add(Dense(units=6))
model.add(Activation('softmax')) # SoftMax activation needs to be separate from Dense to remove it later on
# EXPLORE Learning Rate
opt = tf.keras.optimizers.Adam(learning_rate=10e-3)
model.summary()
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

## Train model

In [ ]:
model.fit(x_train, y_train, epochs=3, validation_data=(x_test, y_test))

## Evaluate model on complete test dataset

In [ ]:
model.evaluate(x_test, y_test, verbose=2)
pred_test = model.predict(x_test)
print(tf.math.confusion_matrix(y_test.argmax(axis=1), pred_test.argmax(axis=1)))

## Evaluate model on small test dataset

In [ ]:
model.evaluate(x_test_250, y_test_250, verbose=2)
pred_test_250 = model.predict(x_test_250)
print(tf.math.confusion_matrix(y_test_250.argmax(axis=1), pred_test_250.argmax(axis=1)))

## Save trained model

In [ ]:
model.save('lab1_part2_uci-har.h5')